In [1]:
import numpy as np

'''
데이터 생성
'''
rng = np.random.RandomState(123)

d = 2
N = 10

mean = 5

x1 = rng.randn(N, d) + np.array([0, 0])
x2 = rng.randn(N, d) + np.array([mean, mean])

x = np.concatenate((x1, x2), axis=0)

'''
단순 퍼셉트론
'''
w = np.zeros(d)
b = 0


def y(x):
    return step(np.dot(w, x) + b)


def step(x):
    return 1 * (x > 0)


def t(i):
    if i < N:
        return 0
    else:
        return 1


while True:
    classified = True
    for i in range(N * 2):
        delta_w = (t(i) - y(x[i])) * x[i]
        delta_b = (t(i) - y(x[i]))
        w += delta_w
        b += delta_b
        classified *= all(delta_w == 0) * (delta_b == 0)
    if classified:
        break

print('w:', w)
print('b:', b)

print('\nTest:')
print(y([0, 0]))  # => 0
print(y([5, 5]))  # => 1


w: [2.14037745 1.2763927 ]
b: -9

Test:
0
1


In [4]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 


'''
모델 설정
'''
tf.set_random_seed(0)  # 난수 시드

w = tf.Variable(tf.zeros([2, 1]))
b = tf.Variable(tf.zeros([1]))

x = tf.placeholder(tf.float32, shape=[None, 2])
t = tf.placeholder(tf.float32, shape=[None, 1])
y = tf.nn.sigmoid(tf.matmul(x, w) + b)

cross_entropy = - tf.reduce_sum(t * tf.log(y) + (1 - t) * tf.log(1 - y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.to_float(tf.greater(y, 0.5)), t)

'''
모델 학습
'''
# OR 게이트
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
Y = np.array([[0], [1], [1], [1]])

# 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# 학습
for epoch in range(200):
    sess.run(train_step, feed_dict={
        x: X,
        t: Y
    })

'''
학습 결과를 확인한다
'''
classified = correct_prediction.eval(session=sess, feed_dict={
    x: X,
    t: Y
})
prob = y.eval(session=sess, feed_dict={
    x: X
})

print('classified:')
print(classified)
print()
print('output probability:')
print(prob)


Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Use `tf.cast` instead.
classified:
[[ True]
 [ True]
 [ True]
 [ True]]

output probability:
[[0.22355038]
 [0.9142595 ]
 [0.9142595 ]
 [0.9974742 ]]


In [5]:
import numpy as np
#import tensorflow as tf
from sklearn.utils import shuffle

np.random.seed(0)
tf.set_random_seed(0)

M = 2      # 입력 데이터의 차원
K = 3      # 클래스 수
n = 100    # 각 클래스에 있는 데이터 수
N = n * K  # 전체 데이터 수

'''
데이터 생성
'''
X1 = np.random.randn(n, M) + np.array([0, 10])
X2 = np.random.randn(n, M) + np.array([5, 5])
X3 = np.random.randn(n, M) + np.array([10, 0])
Y1 = np.array([[1, 0, 0] for i in range(n)])
Y2 = np.array([[0, 1, 0] for i in range(n)])
Y3 = np.array([[0, 0, 1] for i in range(n)])

X = np.concatenate((X1, X2, X3), axis=0)
Y = np.concatenate((Y1, Y2, Y3), axis=0)

'''
모델 생성
'''
W = tf.Variable(tf.zeros([M, K]))
b = tf.Variable(tf.zeros([K]))

x = tf.placeholder(tf.float32, shape=[None, M])
t = tf.placeholder(tf.float32, shape=[None, K])
y = tf.nn.softmax(tf.matmul(x, W) + b)

cross_entropy = tf.reduce_mean(-tf.reduce_sum(t * tf.log(y),
                               reduction_indices=[1]))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))

'''
모델을 학습시킨다
'''
# 초기화
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

batch_size = 50  # 미니배치의 크기
n_batches = N // batch_size

# 미니배치 학습
for epoch in range(20):
    X_, Y_ = shuffle(X, Y)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end]
        })

'''
학습 결과를 확인한다
'''
X_, Y_ = shuffle(X, Y)

classified = correct_prediction.eval(session=sess, feed_dict={
    x: X_[0:10],
    t: Y_[0:10]
})
prob = y.eval(session=sess, feed_dict={
    x: X_[0:10]
})

print('classified:')
print(classified)
print()
print('output probability:')
print(prob)


classified:
[ True  True  True  True  True  True  True  True  True  True]

output probability:
[[9.9868268e-01 1.3173160e-03 2.5878538e-10]
 [3.4313095e-03 9.6904927e-01 2.7519440e-02]
 [9.6915740e-01 3.0842537e-02 5.0909865e-08]
 [1.9351479e-02 9.7068453e-01 9.9640097e-03]
 [4.1215809e-09 8.5910317e-03 9.9140900e-01]
 [1.7154520e-02 9.7633582e-01 6.5097027e-03]
 [2.3086071e-07 4.2457711e-02 9.5754212e-01]
 [7.2534569e-08 8.8783968e-03 9.9112153e-01]
 [8.4302730e-12 1.4183836e-04 9.9985814e-01]
 [9.9589419e-01 4.1058036e-03 2.6871794e-09]]
